# Prepare Gibbs-Duhem Simulations

In [1]:
import os, sys 
import numpy as np

In [2]:
settings=dict()

settings['--step']=250
settings['--initial_equilibration_steps']=500000
settings['--steps_per_sim']=250000

settings['--percent_equilibration']=25.0
settings['--lmp_exe']='$LAMMPS_INTEL'
settings['--run_cmd']='"export OMP_NUM_THREADS=2;srun -n 64"'
settings['--lmp_options']='" -sf omp -partition 2x32 "'

settings['--root_fold']='./'

settings_T=settings.copy()
settings_T['--integration_variable']= 'T'
settings_T['--step']=5


In [3]:
def PrepSingleSim(setttings_temp,dir_name,left_phase,right_phase,pm,tm,pmin,pmax):
    dir_name=dir_name+'/'
    # Create simulation dir
    root_folder=dir_name+'/'+str(pm)+'atm'+'_'+str(tm)+'K'
    os.makedirs(dir_name, exist_ok=True)#s.system('mkdir  {}'.format(dir_name))
    os.makedirs(dir_name+'/Initial', exist_ok=True)
    os.makedirs(root_folder, exist_ok=True)

    os.system('cp integrate.py ' + dir_name)
    initial='Initial/'
    os.system('cp -r InitialConfigurations/{} {}{}'.format(left_phase,dir_name, initial) )
    os.system('cp -r InitialConfigurations/{} {}{}'.format(right_phase,dir_name, initial))

    
    
    # Setup command for running

    setttings_temp['--end_variable']=str(pmax)
    setttings_temp['--initial_point_folder']='../'+initial
    setttings_temp['--left']=left_phase
    setttings_temp['--right']=right_phase
    setttings_temp['--initial_TP']=str(tm)+' '+str(pm) 
    arguments=' '.join(['{} {}'.format(t,setttings_temp[t]) for t in setttings_temp])
    
    # Upwards
    arguments=' '.join(['{} {}'.format(t,setttings_temp[t]) for t in setttings_temp])
    command = 'python3 ../integrate.py {}\n'.format(arguments) 
    
    #Downwards
    setttings_temp['--end_variable']=str(pmin)
    arguments=' '.join(['{} {}'.format(t,setttings_temp[t]) for t in setttings_temp])
    command += 'python3 ../integrate.py {}\n'.format(arguments) 
    
    with open(root_folder+'/script.sh','w') as fp:
        fp.write(command)
    

### IceIh-Liquid

In [11]:
for PT in np.loadtxt('../IceIh-Liquid/4-BiasedCoexistence/P_T.dat'):
    print(PT)
    PrepSingleSim(settings,'IceIh_Liquid/','IceIh','Liquid',PT[0],PT[1],-5000,4000)

[  0.  269.5]
[1000.   259.3]
[2000.   246.3]
[2500.   236.5]
[3000.   217.3]


## IceII-Liquid

In [ ]:
for PT in np.loadtxt('../IceII_Liquid/5-BiasedCoexistence/P_T.dat'):
    print(PT)

    PrepSingleSim(settings,'IceII_Liquid/','IceII','Liquid',PT[0],PT[1],-2000,8000)

## IceIII-Liquid

In [ ]:
for PT in np.loadtxt('../IceIII-Liquid/5-BiasedCoexistence/P_T.dat'):
    print(PT)
    PrepSingleSim(settings,'IceIII_Liquid/','IceIII','Liquid',PT[0],PT[1],-1000,12000)

## IceV-Liquid

In [ ]:
for PT in np.loadtxt('../IceV-Liquid/4-BiasedCoexistence/P_T.dat'):
    print(PT)
    PrepSingleSim(settings,'IceV_Liquid/','IceV','Liquid',PT[0],PT[1],2000,12000)

## IceVI-Liquid

In [6]:
for PT in np.loadtxt('../IceVI-Liquid/5-BiasedCoexistence/P_T.dat'):
    print(PT)
    PrepSingleSim(settings,'IceVI_Liquid/','IceVI','Liquid',PT[0],PT[1],6000,15000)

[8000.   259.2]
[9000.   266.6]
[10000.    274.8]
[12000.    286.9]


## IceVI-IceV

In [13]:
with open('../Full_Phase_Diagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceV IceVI' in line:
            PrepSingleSim(settings_T,'IceVI_IceV/','IceVI','IceV',float(PT[-1]),float(PT[-2]),50,280)

## IceIII IceV

In [14]:
with open('../Full_Phase_Diagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceIII IceV' in line:
            PrepSingleSim(settings_T,'IceV_IceIII/','IceV','IceIII',float(PT[-1]),float(PT[-2]),50,280)

## IceII IceIh

In [15]:
with open('../Full_Phase_Diagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceIh IceII' in line:
            PrepSingleSim(settings_T,'IceII_IceIh/','IceII','IceIh',float(PT[-1]),float(PT[-2]),0,250)

cp: InitialConfigurations/IceII/.#in.dump: No such file or directory


## IceIII IceIh

In [16]:
with open('../Full_Phase_Diagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'Liquid IceIII IceIh' in line:
            PrepSingleSim(settings_T,'IceIh_IceIII/','IceIh','IceIII',float(PT[-1]),float(PT[-2]),0,250)

## IceII IceIII

In [4]:
with open('../Full_Phase_Diagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'IceII IceIh IceIII' in line:
            PrepSingleSim(settings,'IceII_IceIII/','IceII','IceIII',float(PT[-1]),float(PT[-2]),12000,-2000)

cp: InitialConfigurations/IceII/.#in.dump: No such file or directory


## IceII IceV

In [4]:
with open('../Full_Phase_Diagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'IceII IceIII IceV' in line:
            PrepSingleSim(settings,'IceII_IceV/','IceII','IceV',float(PT[-1]),float(PT[-2]),15000,7000)

cp: InitialConfigurations/IceII/.#in.dump: No such file or directory


## IceII IceVI

In [5]:
with open('../Full_Phase_Diagram/triple_points.dat','r') as fp:
    lines=fp.readlines()
    
    for line in lines:
        PT=line.split()
        if 'IceII IceV IceVI' in line:
            PrepSingleSim(settings_T,'IceII_IceVI/','IceII','IceVI',float(PT[-1]),float(PT[-2]),120,0)

cp: InitialConfigurations/IceII/.#in.dump: No such file or directory


In [ ]:
!jupyter nbconvert --to script PrepareGibbsDuhem.ipynb